# 네이버 카페 크롤링

## import

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup
from collections import defaultdict
import time
import csv
import pandas as pd
import numpy as np
import requests               
import lxml

## chrome option

In [2]:
options = ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging']) # Usb Error ignore
options.add_argument("no-sandbox") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('user-agent='+ user_agent)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [23]:
driver = webdriver.Chrome(options=options)

url = 'https://cafe.naver.com/vilab/ArticleList.nhn?search.clubid=11525920&search.boardtype=L'
driver.get(url)

driver.implicitly_wait(10)
driver.switch_to.frame("cafe_main")
print("wake up")

time.sleep(1)
naver_data = defaultdict(list)

wake up


## Title, Date, Link

In [ ]:
driver.switch_to.frame("cafe_main")

while(1):
    href_list = []
    button_table = driver.find_element(By.XPATH, "//*[@id=\"main-area\"]/div[6]")
    buttons = button_table.find_elements(By.TAG_NAME,"a")

    for button_link in buttons:
        blink = button_link.get_attribute('href')
        href_list.append(blink)
    print(href_list)
    for i in range(len(buttons)):
        # button이 숫자가 아니면
        #print(buttons[i].text)
        if not (buttons[i].text.isdigit()):
            continue
        # page에 들어가기
        driver.get(href_list[i])
        time.sleep(2)
        driver.implicitly_wait(10)
        driver.switch_to.frame('cafe_main')
        # 2023년인지 확인
        cut = driver.find_element(By.XPATH, "//*[@id=\"main-area\"]/div[4]/table/tbody/tr[1]/td[3]").text
        if "2022" in cut:
            a = False
            break
        
        #조회수
        num_clicks = driver.find_elements(By.CLASS_NAME,"td_view")
        for num_click in num_clicks:
            naver_data['click'].append(num_click.text)

        
        #날짜
        dates = driver.find_elements(By.CLASS_NAME,"td_date")
        for date in dates:
            naver_data['date'].append(date.text)
    
        #제목이랑 link 뽑기
        titles = driver.find_elements(By.CLASS_NAME,"article")
        for title in titles:
            href = title.get_attribute('href')
        
            naver_data['title'].append(title.text)
            naver_data['link'].append(href)

        button_table = driver.find_element(By.XPATH, "//*[@id=\"main-area\"]/div[6]")
        buttons = button_table.find_elements(By.TAG_NAME,"a")

    next = driver.find_element(By.CLASS_NAME,'pgR')
    next.click()
    if a == False:
        break

min_length = min(len(lst) for lst in naver_data.values())
trimmed_data = {key: lst[:min_length] for key, lst in naver_data.items()}
df = pd.DataFrame(trimmed_data)
df.to_csv("naver_data1.csv", index = False)

## Content, Comment

In [24]:
naver_data = pd.read_csv("naver_data1.csv")
naver_data2 = defaultdict(list)

In [ ]:
#driver.switch_to.frame('cafe_main')
for link in naver_data['link']:
    try:
        driver.get(link)
        time.sleep(1)
        driver.switch_to.frame('cafe_main')
        
        req = driver.page_source
        driver.implicitly_wait(10)
        soup = BeautifulSoup(req, 'lxml')
    except:
        naver_data2['content'].append(None)
        naver_data2['comment'].append(None)
        continue

    try:
        content = soup.select(".article_viewer")

    except:
        naver_data2['content'].append(None)

        continue

    try:
        comment = soup.select('.comment_text_view')

    except:
        naver_data2['comment'].append(None)
        continue

    text_comment = ''
    text_content = ''
 
    for c in comment:
        text_comment  = text_comment +'\n' +  c.text
    
    for c in content:
        text_content  = text_content +'\n' +  c.text

    print(text_comment, text_content)
    
    naver_data2['content'].append(text_content)
    naver_data2['comment'].append(text_comment)



In [26]:
min_length = min(len(lst) for lst in naver_data2.values())
trimmed_data = {key: lst[:min_length] for key, lst in naver_data2.items()}
df = pd.DataFrame(trimmed_data)
df.to_csv("naver_data_content.csv", index = False)